In [26]:
from numba import cuda
import numpy as np 
import math
import random

In [27]:
n = 100000
k = 64
LARGE = 9999

In [28]:
X = np.array(np.random.random((n, k)), dtype=np.float32)

In [29]:
np.random.seed(39)
for i in range(n):
    for j in range(k):
        X[i, j] = np.random.uniform()

In [30]:
X = np.array(X)

In [31]:
X.shape

(100000, 64)

In [32]:
print(X)

[[0.5468892  0.797899   0.8204019  ... 0.89582473 0.8865771  0.01363686]
 [0.8688695  0.30145565 0.94794893 ... 0.17431284 0.56729794 0.9840169 ]
 [0.52914655 0.43819624 0.91374445 ... 0.27539888 0.67386234 0.7993358 ]
 ...
 [0.45885056 0.5653358  0.6383706  ... 0.16754374 0.5548967  0.5647587 ]
 [0.702312   0.75162303 0.07410108 ... 0.46562812 0.52337885 0.14129098]
 [0.5299121  0.68515706 0.8040761  ... 0.8194798  0.16229047 0.68227106]]


In [33]:
# f= open("valX.txt","w")

In [34]:
'''
for i in range(n):
    f.write("[")
    for j in range(k):
        if(j!=(k-1)):
            f.write(str(X[i,j])+",")
        else:
            f.write(str(X[i,j]))
    f.write("],\n")
f.close()
'''

'\nfor i in range(n):\n    f.write("[")\n    for j in range(k):\n        if(j!=(k-1)):\n            f.write(str(X[i,j])+",")\n        else:\n            f.write(str(X[i,j]))\n    f.write("],\n")\nf.close()\n'

In [35]:
@cuda.jit('void(float32[:,:],float32[:,:])')
def cuda_mult(X,res):
    """This kernel function will be executed by a thread."""
    row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y;
    col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x;
    if ((row >= n) | (col >=n )):
        return
    temp_sum = 0.0
    for i in range(n):
        temp_sum += X[row,i] * X[i,col]
    res[row,col] = temp_sum

In [36]:
#@cuda.jit('void(float32[:,:],float32[:,:],float32[:,:],float32[:,:],float32[:,:],float32[:,:],float32[:,:])')
@cuda.jit(max_registers=63) 
def cuda_dist(X,first_val,first_index,second_val,second_index,third_val,third_index):
    SMALL = -9999.0
    """This kernel function will be executed by a thread."""
    row = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y;
    col = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x;
    if ((row >= n) | (col >=k )):
        return
    if(col==0):
        first_val[row,col] = SMALL
        first_index[row,col] = -1
        second_val[row,col] = SMALL
        second_index[row,col] = -1
        third_val[row,col] = SMALL
        third_index[row,col] = -1
        for i in range(n):
            tmp = 0.0
            magnitude1 = 0.0
            magnitude2 = 0.0
            for j in range(k):
                tmp += X[row,j] * X[i,j]
                magnitude1 += (X[row,j]* X[row,j])
                magnitude2 += (X[i,j]* X[i,j])
            tmp /= (math.sqrt(magnitude1)*math.sqrt(magnitude2))
            if(tmp>=first_val[row,col]):
                third_val[row,col] = second_val[row,col]
                third_index[row,col] = second_index[row,col]
                second_val[row,col] = first_val[row,col]
                second_index[row,col] = first_index[row,col]
                first_val[row,col] = tmp
                first_index[row,col] = i
            elif(tmp>=second_val[row,col]):
                third_val[row,col] = second_val[row,col]
                third_index[row,col] = second_index[row,col]
                second_val[row,col] = tmp
                second_index[row,col] = i
            elif(tmp>third_val[row,col]):
                third_val[row,col] = tmp
                third_index[row,col] = i

In [37]:
device = cuda.get_current_device()

In [38]:
device.WARP_SIZE 

32

In [39]:
d_x = cuda.to_device(X)
d_first_val = cuda.device_array_like(d_x)
d_first_index = cuda.device_array_like(d_x)
d_second_val = cuda.device_array_like(d_x)
d_second_index = cuda.device_array_like(d_x)
d_third_val = cuda.device_array_like(d_x)
d_third_index = cuda.device_array_like(d_x)

tpb = device.WARP_SIZE       #blocksize or thread per block
bpg = int(np.ceil((n)/tpb))  #block per grid


In [40]:
tpb, bpg, tpb*bpg

(32, 3125, 100000)

In [41]:
%time cuda_dist[(bpg, bpg),(tpb, tpb)](d_x,d_first_val,d_first_index,d_second_val,d_second_index,d_third_val,d_third_index)



CPU times: user 427 ms, sys: 0 ns, total: 427 ms
Wall time: 453 ms


In [42]:
# Transfer output from device to host
first_val = d_first_val.copy_to_host()
print (first_val[:,0])

[1. 1. 1. ... 1. 1. 1.]


In [43]:
# Transfer output from device to host
first_index = d_first_index.copy_to_host()
print (first_index[:,0])

[0.0000e+00 1.0000e+00 2.0000e+00 ... 9.9997e+04 9.9998e+04 9.9999e+04]


In [44]:
# Transfer output from device to host
second_val = d_second_val.copy_to_host()
print (second_val[:,0])

# Transfer output from device to host
second_index = d_second_index.copy_to_host()
print (second_index[:,0])

[0.91594034 0.89068705 0.89246786 ... 0.8838843  0.8867385  0.8910104 ]
[59243.  1885. 65836. ... 15520. 75121.  1471.]


In [45]:
# Transfer output from device to host
third_val = d_third_val.copy_to_host()
print (third_val[:,0])

# Transfer output from device to host
third_index = d_third_index.copy_to_host()
print (third_index[:,0])

[0.90159434 0.89040256 0.89246047 ... 0.88327575 0.88625014 0.88911617]
[10844. 36039. 20916. ... 54688. 72831. 71717.]


In [46]:
!nvidia-smi

Mon May  6 06:47:23 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.72       Driver Version: 410.72       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    61W / 300W |   1379MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  